## 내 집 마련을 위해, 서울 아파트 매물 데이터를 이용하여 구매 전략을 세우고자 한다.

real_estate.csv

|컬럼|정의|type|
|:---|:---|:---:|
|latitude|위도|float64|
|longitude|경도|float64|
|price_per_square_py|평당가|float64|
|py|평수|int64|
|apt_code|시공사 코드(길이5 알파벳 대문자)|object|
|dist_from_station|인근 지하철역과의 거리|float64|  

정답 및 해설 : https://tjd229.tistory.com/22

In [17]:
import numpy as np
import pandas as pd

df = pd.read_csv('../content/tjd229/real_estate.csv')

### Q1. 시공사별 아파트 평수의 평균을 구하려고 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.

단계 1 : 데이터의 첫 번째 행부터 열 번째 행까지의 시공사 코드(apt_code) 리스트를 저장한다.  
첫 번째 행의 데이터는 위도, 경도, 평당가, 평수, 시공사 코드, 인근 지하철역과의 거리가 각각 37.125541, 126.913776, 2860.053787, 24, GEDAE, 1588.406226이다.  
단계 2 : 시공사 코드(apt_code)가 단계 1에서 구한 리스트에 포함되어 있는 데이터를 전부 제거한다.  
단계 3 : 시공사 코드(apt_code)별 평수 평균을 구하고 그 중 가장 큰 값을 구하시오  

※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.
(정답 예시: 2.29)

In [18]:
df1 = df.copy()
_remove_aptcode = df1.head(10).apt_code.tolist()
_remove_aptcode

['GEDAE',
 'IHEAD',
 'FHIBJ',
 'JBCIJ',
 'EDJCI',
 'EHCAD',
 'HEBIA',
 'HEBIA',
 'IIGFI',
 'FHIBJ']

isin()

In [19]:
df1_remove = df1.loc[~df1.apt_code.isin(_remove_aptcode)].copy()
df1_remove

,latitude,longitude,price_per_square_py,py,apt_code,dist_from_station
10,37.046400,126.837832,3750.582844,40,GBDGG,1493.140743
11,37.215855,127.127967,1945.709193,32,HCCBB,989.515201
12,37.252653,127.107425,2670.280398,32,IDCIC,1802.915110
13,37.132489,126.784801,2246.292722,40,HCCBB,922.157169
14,37.270844,127.064936,2423.397245,24,EGFBJ,1491.398873
...,...,...,...,...,...,...
770,37.251483,127.073069,2356.504401,63,HHGCB,1416.903407
771,37.078108,126.723392,1884.239692,24,DEACF,967.757381
773,37.288945,127.118062,3072.810919,24,GBDGG,1803.257080
774,37.234408,126.745186,2491.714188,61,EJAIA,1346.821965


In [20]:
df1_remove.groupby('apt_code').mean().max().loc['py'].round(2)

38.63

### Q2. 아파트 매물을 분석하기 위해 군집화를 하려고 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.


단계 1 : 독립 변수들에 대해 K-means 군집 분석을 수행한다. 이 때, 군집 수는 2~9개 중 K-means Silhouette 를 통해 구하고, 이 중 첫번째로 높은 score를 최적의 K로 설정한다.  
            - 독립 변수(총 5개) : 시공사 코드(apt_code)를 제외한 모든 컬럼  


#### 필요 라이브러리 함수,클래스 및 설정값 목록  


from sklearn.cluster import KMeans  
from sklearn.metrics import silhouette_score    
random_state = 229  
문제 지시 외 Default 값 사용  

In [21]:
df1_remove.columns

Index(['latitude', 'longitude', 'price_per_square_py', 'py', 'apt_code',
       'dist_from_station'],
      dtype='object')

In [22]:
X_train = df1_remove[['latitude', 'longitude', 'price_per_square_py', 'py', #'apt_code',
       'dist_from_station']]


In [23]:
df1_remove.shape

(552, 6)

👉 silhouette

In [24]:
from sklearn.cluster import KMeans  
from sklearn.metrics import silhouette_score

K = range(2,10)
sil_list= []

for k in K :
    sil = KMeans(n_clusters=k, init='k-means++', n_init=10, random_state=229)
    sil.fit(X_train)
    sil_list.append(silhouette_score(X_train,sil.labels_))

sil_list


[0.631203691848422,
 0.4470936176679579,
 0.45560050005825237,
 0.43495847469472787,
 0.3884871247601564,
 0.39086044963336697,
 0.37140972679517836,
 0.3704764809515313]

In [25]:
sil_df = pd.DataFrame({ "K":K,
                      "sil_coef": sil_list})

max_sil_k = sil_df.sort_values(by='sil_coef',ascending=False).iat[0,0]

# sil coef 값은 
# max_sil_k = sil_df.sort_values(by='sil_coef',ascending=False).iat[0,1]

👉 KMean

In [26]:


mykm = KMeans(n_clusters=max_sil_k, init='k-means++', n_init=10, random_state=229)
mykm.fit(X_train)
df1_remove['cluster'] = mykm.labels_



In [27]:

df1_remove['cluster'].value_counts(normalize=True).iat[0]


0.7355072463768116


단계 2 : 단계 1에서 최적의 K로 도출한 각 군집의 비율 중, 가장 큰 값을 구하시오  

※ 정답은 올림하여 소수점 둘째 자리까지 출력하시오.
(정답 예시: 2.29)

### Q3. 아파트 가격을 예측하는 모델을 만드려고 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.

단계 1 : 평당가(price_per_square_py)와 평수(py)를 곱하여 실제 가격 변수를 생성한다.  
단계 2 : Train Set과 Test Set을 28/30:2/30 비율로 나눈다.  


#### 필요 라이브러리 함수,클래스 및 설정값 목록  


from sklearn.linear_model import LinearRegression  
from sklearn.model_selection import train_test_split  
모든 random_state = 229  
문제 지시 외 Default 값 사용  

In [28]:
df1_real = df.copy()

In [29]:
df1_real['rea_val'] = df1_real.price_per_square_py * df1_real.py
df1_real.head()

,latitude,longitude,price_per_square_py,py,apt_code,dist_from_station,rea_val
0,37.125541,126.913776,2860.053787,24,GEDAE,1588.406226,68641.290895
1,37.223169,127.026913,1715.073163,59,IHEAD,986.146800,101189.316628
2,37.239029,126.906303,1780.604630,57,FHIBJ,1492.514512,101494.463921
3,37.274433,126.737771,1508.814336,42,JBCIJ,1621.602866,63370.202096
4,37.082670,127.143634,2933.930488,32,EDJCI,945.034519,93885.775621


df.drop()

https://wikidocs.net/154050  

df.drop(labels=None, axis=0, index=None, columns=None, level=None, inplace=False, errors='raise')  
labels : 삭제할 레이블명입니다. axis를 지정해주어야합니다.  
axis : {0 : index / 1 : columns} labels인수를 사용할경우 지정할 축입니다.  
index : 인덱스명을 입력해서 바로 삭제를 할 수 있습니다.  
columns : 컬럼명을 입력해서 바로 삭제를 할 수 있습니다.  
level : 멀티인덱스의 경우 레벨을 지정해서 진행할 수 있습니다.  
inplace : 원본을 변경할지 여부입니다. True일경우 원본이 변경됩니다.  
errors : 삭제할 레이블을 찾지 못할경우 오류를 띄울지 여부입니다. ignore할 경우 존재하는 레이블만 삭제됩니다.  
※ axis=0 + labels 는 index인수와 역할이 같고 axis=1 + labels는 columns와 역할이 같습니다.   

In [30]:
df1_real.drop('apt_code',axis=1, inplace=True)
df1_real.tail()

,latitude,longitude,price_per_square_py,py,dist_from_station,rea_val
772,37.161852,126.917406,3564.856667,40,1593.663253,142594.266660
773,37.288945,127.118062,3072.810919,24,1803.257080,73747.462061
774,37.234408,126.745186,2491.714188,61,1346.821965,151994.565484
775,37.294494,127.185185,1746.372819,40,1461.672552,69854.912763
776,37.004345,127.107611,1667.553347,32,1377.100866,53361.707088


In [31]:
from sklearn.linear_model import LinearRegression  
from sklearn.model_selection import train_test_split

👉 데이터셋 분리

일변일 경우에도 리스트로 전달  :  X = df1_real[['latitude']]

In [32]:
# 독립변수 
X = df1_real[['latitude','longitude','price_per_square_py','dist_from_station','py']]

# 종속변수
y = df1_real['rea_val']

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=2/30, random_state=229)

단계 3 : Train Set으로 LinearRegression 모델을 학습하고, Test Set에 적용한다.   
이 때, 절편을 포함한 경우와 포함하지 않은 경우 각각에 대하여 수행한다.  
  
            - 독립 변수(총 5개) : 시공사 코드(apt_code)와 총 가격을 제외한 모든 컬럼  
            - 종속 변수 : 실제 가격  

단계 3에서 학습한 각 모델에 대하여 예측 결과를 아래 정의된 Measure M으로 계산하였을 때 가장 큰 값은?  
$$ M = \biggl(\frac{1}{n} \sum_{i=1}^{n}{(y_i - \hat{y_i})^2}\biggr), \quad \hat{y_i}: 예측값, y_i: 실제값 $$

※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.
(정답 예시: 2.29)

👉 (다중) 선형 회기

In [34]:
real_reg_mo = LinearRegression()
real_reg_mo.fit(X_train, y_train)
y_pred = real_reg_mo.predict(X_test)
y_pred

array([ 71537.06357488,  37200.1213293 ,  35494.01255878,  70471.6408111 ,
        66383.60406998,  75463.1405433 ,  78045.00045754,  58905.29454575,
        65828.22622791,  75100.79955153, 106680.98678017,  78651.20939828,
        61362.85360125,  32623.15234517,  61709.96732249,  36039.69474832,
        61505.47616961,  72107.82903833,  45400.45765664,  68439.60425606,
        72205.20891592,  77442.4900336 , 126070.02549163,  71156.01035643,
        76937.07422418,  80012.34903584,  58971.82398002,  36522.59704217,
        54186.30516848,  51968.30500402,  96691.9974597 , 107056.99700545,
        56149.37950605,  40722.00357319, 155673.66616875,  46700.36282707,
        31876.42694448, 100071.39018113,  38681.4786977 ,  79690.50844088,
        28663.71183769,  50830.05911825,  69194.33878661,  50121.74747649,
        64424.35592352,  68546.3237267 ,  96880.62501702,  95966.21995102,
        68226.19396395,  32537.41804298,  30538.15639693,  76144.92179301])

In [35]:
# 절편이 없는 경우 

real_reg_mo_nointer = LinearRegression(fit_intercept=False)
real_reg_mo_nointer.fit(X_train, y_train)
y_pred_nointer= real_reg_mo_nointer.predict(X_test)
y_pred_nointer

array([ 71427.99389131,  37340.94086899,  35672.38701553,  70390.85330768,
        66588.3801655 ,  75637.33514206,  78237.96205798,  58891.31154552,
        65824.19984416,  75062.4501454 , 106588.90904681,  78579.09045147,
        61516.43389792,  32522.31680278,  61548.54917857,  35972.70060157,
        61599.41435747,  72075.78054911,  45387.05849864,  68426.17098315,
        72201.54983459,  77219.27325969, 125996.43040523,  71214.61840311,
        76707.17492305,  79711.06928229,  58740.54046661,  36453.66058087,
        54334.38241459,  51921.43732011,  96589.6611447 , 107070.36789877,
        56117.52136343,  40769.72426759, 155806.03911506,  46767.95139621,
        31790.48903619, 100418.15476958,  38810.68756198,  79864.38542935,
        28337.99968621,  50653.12697544,  69400.8622678 ,  49904.75210801,
        64288.85849315,  68427.96624782,  96788.24991984,  95832.33827934,
        68329.45721146,  32608.21481703,  30683.22262999,  76166.93103478])

In [36]:
# 절편이 없는 경우, MSE 의 실제 계산
res = (((y_pred_nointer - y_test)**2).mean())
res

19007465.6748951

- coefficient 의 의미

In [37]:
real_reg_mo.coef_

# 차례대로 'latitude','longitude','price_per_square_py','dist_from_station','py' 에 대한 기울기.

array([-6.47134835e+02,  8.80496226e+02,  3.50347152e+01,  8.10706193e-01,
        2.21214720e+03])

In [38]:
real_reg_mo.intercept_

-166330.05711403547

### 👉 (다중) 회귀모델 평가
1. MAE (Mean Absolute Error) : (실제값과 예측값) 차이의 절대값
2. MSE (Mean SquareEd Error) : 차이의 제곱
3. RMSE (Root Mean Squared Error) : 차이의 제곱의 루트
4. MAPE (Mean Absolute Percentage Error) : 평균 절대 비율 오차 = np.mean(np.abs(y_test - pred / y_test))*100  
5. R2 : 결정계수  (1 - SSE/SST)
6. Adjusted-R2
    
> R2 는 1에 가까울수록, 나머지는 0에 가까울수록 좋음

In [39]:
# 훈련 데이터 평가

# 복수개의 train 데이터 일때 각 데이터벌 r2_score 가 좋은 (1 가까울수록 ) 데이터 셋 찾기

real_reg_mo.score(X_train,y_train)

0.953888849310125

In [40]:
# 테스트 데이터 평가  = r2_score(y_test, y_pred)

from sklearn.metrics import r2_score

real_reg_mo.score(X_test,y_test), r2_score(y_test, y_pred)

(0.9689589651349582, 0.9689589651349582)

1. MAE

In [41]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

# y_test : 실제값
# y_pred : 모델로 예측한 값

3234.468890751421

2. MSE

In [42]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,y_pred)

19007139.276858456

In [43]:
# 절편이 없는 경우
mean_squared_error(y_test,y_pred_nointer)

# 실제 계산 값 46159607.7447345 과 동일

19007465.6748951

3. RMSE

In [44]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred, squared =False)

4359.717797846376

4. MPAE

In [45]:
MPAE = np.mean(np.abs((y_test-y_pred)/y_test))* 100

MPAE

5.673867287162161

In [46]:
from sklearn.metrics import mean_absolute_percentage_error

mean_absolute_percentage_error(y_test,y_pred)

0.05673867287162161

5. R2 

** 결과 값이 model.score(X_test, y_test) 와 같다. **

In [47]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.9689589651349582

6. A-R2

n : 데이터 갯수 (row 갯수 : train? test?)
p : features 갯수 (X column 갯수)

Adjusted R2 = 1 - (1-model.score(X_test, y_test))*(len(y_test_)-1)/(len(y_test)-X_test.shape[1]-1)


In [48]:
len(y_test), X_test.shape[1]

(52, 5)

In [49]:
# model.score(X_test,y_test) 사용

A_R2 = 1 - (1-real_reg_mo.score(X_test, y_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
A_R2




0.9655849396061492

In [50]:
# r2_score(y_test, y_pred) 사용

A_R2 = 1 - (1-r2_score(y_test, y_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
A_R2

0.9655849396061492

In [51]:
len(y_train), X_train.shape[1]

(725, 5)

In [52]:

# train 데이터들의 A_r2 값을 찾으려면, X_train, y_train 을 넣어야 하나? 
# model.score(X_train, y_train)

A_R2 = 1 - (1-real_reg_mo.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
A_R2

0.9535681876224346

### 👉 OLS, ols  

https://blog.naver.com/greenysky/223095471469  

OLS
```
from statsmodels.api import add_constant 
from statsmodels.api import OLS 

X_train_ad = add_constant(X_train)    

myOLS=OLS(y_train, X_train_ad).fit()  
myOLS.summary()  

# import statsmodels.api as sm
# X_train_ad = sm.add_constant(X_train)
# myOLS = sm.OLS(y_train, X_train_add)
```

ols
```
from statsmodels.formula.api import ols

train.columns = [ y_col, X_col_1 ... , X_col_end ]

formula = 'y_col ~ X_col_1 + X_col_2 + X_col_3+ ... + X_col_end'  
my_ols = ols(formula = formula, data=train).fit()  
my_ols.summary()  
```

In [66]:
from statsmodels.api import add_constant  
X_train_ad = add_constant(X_train)  
X_test_ad = add_constant(X_test)  
print(X_train_ad) 

     const   latitude   longitude  price_per_square_py  dist_from_station  py
301    1.0  37.055723  126.885253          1569.575556        1464.859132  40
325    1.0  37.170299  126.758981          1631.638340        1371.082457  32
178    1.0  37.051574  126.755140          3350.798249        1669.022820  24
443    1.0  37.026883  126.921716          1665.869452         935.042689  24
109    1.0  37.239580  127.094128          1749.377651        1865.419774  40
..     ...        ...         ...                  ...                ...  ..
501    1.0  37.062698  127.047770          1895.194907        1509.302856  32
112    1.0  37.101601  126.920339          1575.314791        1345.840700  40
680    1.0  37.289144  126.736108          1874.936004        1516.976770  32
476    1.0  37.019957  127.140616          1856.520904        1225.581824  24
345    1.0  37.064679  127.132237          3593.978646        1632.240908  40

[725 rows x 6 columns]


In [79]:
from statsmodels.api import OLS  
# import statsmodels.api as sm

myOLS=OLS(y_train, X_train_ad).fit()     # myOLS = OLS() 와 표현이 다른점에 주의 
myOLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                rea_val   R-squared:                       0.954
Model:                            OLS   Adj. R-squared:                  0.954
Method:                 Least Squares   F-statistic:                     2975.
Date:                Sat, 24 Jun 2023   Prob (F-statistic):               0.00
Time:                        20:42:12   Log-Likelihood:                -7499.4
No. Observations:                 725   AIC:                         1.501e+04
Df Residuals:                     719   BIC:                         1.504e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const               -1.663e+05   2.76e+05     -0.602      0.547   -7.09e+05    3.76e+05
latitude             -647.1348   3246.294     -0.199      0.842   -7020.484    5726.214
longitude             880.4962   1944.365      0.453      0.651   -2936.815    4697.808
price_per_square_py    35.0347      0.420     83.320      0.000      34.209      35.860
dist_from_station       0.8107      0.998      0.812      0.417      -1.148       2.770
py                   2212.1472     25.114     88.083      0.000    2162.841    2261.453
==============================================================================
Omnibus:                      325.266   Durbin-Watson:                   1.971
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5411.402
Skew:                           1.576   Prob(JB):                         0.00
Kurtosis:                      16.008   Cond. No.                     2.61e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.61e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [70]:
y_pred_ad = myOLS.predict(X_test_ad)
mean_absolute_error(y_test, y_pred_ad)

# LR MAE : 3234.468890751421

3234.4688907511936

In [85]:
myOLS.params    # output coef


const                 -166330.057114
latitude                 -647.134835
longitude                 880.496226
price_per_square_py        35.034715
dist_from_station           0.810706
py                       2212.147195
dtype: float64

In [59]:
# ols 을 사용하기 위해 train df 생성

train = pd.concat([X_train, y_train],axis=1)
train

,latitude,longitude,price_per_square_py,dist_from_station,py,rea_val
301,37.055723,126.885253,1569.575556,1464.859132,40,62783.022231
325,37.170299,126.758981,1631.638340,1371.082457,32,52212.426875
178,37.051574,126.755140,3350.798249,1669.022820,24,80419.157982
443,37.026883,126.921716,1665.869452,935.042689,24,39980.866852
109,37.239580,127.094128,1749.377651,1865.419774,40,69975.106034
...,...,...,...,...,...,...
501,37.062698,127.047770,1895.194907,1509.302856,32,60646.237018
112,37.101601,126.920339,1575.314791,1345.840700,40,63012.591622
680,37.289144,126.736108,1874.936004,1516.976770,32,59997.952120
476,37.019957,127.140616,1856.520904,1225.581824,24,44556.501698


In [75]:
from statsmodels.formula.api import ols  
formula = 'rea_val ~ latitude+longitude+price_per_square_py+dist_from_station+py'  
my_ols = ols(formula = formula, data=train).fit()  
my_ols.summary() 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                rea_val   R-squared:                       0.954
Model:                            OLS   Adj. R-squared:                  0.954
Method:                 Least Squares   F-statistic:                     2975.
Date:                Sat, 24 Jun 2023   Prob (F-statistic):               0.00
Time:                        19:58:06   Log-Likelihood:                -7499.4
No. Observations:                 725   AIC:                         1.501e+04
Df Residuals:                     719   BIC:                         1.504e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept           -1.663e+05   2.76e+05     -0.602      0.547   -7.09e+05    3.76e+05
latitude             -647.1348   3246.294     -0.199      0.842   -7020.484    5726.214
longitude             880.4962   1944.365      0.453      0.651   -2936.815    4697.808
price_per_square_py    35.0347      0.420     83.320      0.000      34.209      35.860
dist_from_station       0.8107      0.998      0.812      0.417      -1.148       2.770
py                   2212.1472     25.114     88.083      0.000    2162.841    2261.453
==============================================================================
Omnibus:                      325.266   Durbin-Watson:                   1.971
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5411.402
Skew:                           1.576   Prob(JB):                         0.00
Kurtosis:                      16.008   Cond. No.                     2.61e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.61e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [76]:
y_pred_ols = my_ols.predict(X_test)
mean_absolute_error(y_test,y_pred_ols)

# OLS MAE : 3234.4688907511936


3234.468890751195

In [84]:
my_ols.params

Intercept             -166330.057114
latitude                 -647.134835
longitude                 880.496226
price_per_square_py        35.034715
dist_from_station           0.810706
py                       2212.147195
dtype: float64

In [89]:
my_ols2 = ols(formula = formula, data=train)
pd.DataFrame(my_ols2.exog)

,0,1,2,3,4,5
0,1.0,37.055723,126.885253,1569.575556,1464.859132,40.0
1,1.0,37.170299,126.758981,1631.638340,1371.082457,32.0
2,1.0,37.051574,126.755140,3350.798249,1669.022820,24.0
3,1.0,37.026883,126.921716,1665.869452,935.042689,24.0
4,1.0,37.239580,127.094128,1749.377651,1865.419774,40.0
...,...,...,...,...,...,...
720,1.0,37.062698,127.047770,1895.194907,1509.302856,32.0
721,1.0,37.101601,126.920339,1575.314791,1345.840700,40.0
722,1.0,37.289144,126.736108,1874.936004,1516.976770,32.0
723,1.0,37.019957,127.140616,1856.520904,1225.581824,24.0


In [90]:
train

,latitude,longitude,price_per_square_py,dist_from_station,py,rea_val
301,37.055723,126.885253,1569.575556,1464.859132,40,62783.022231
325,37.170299,126.758981,1631.638340,1371.082457,32,52212.426875
178,37.051574,126.755140,3350.798249,1669.022820,24,80419.157982
443,37.026883,126.921716,1665.869452,935.042689,24,39980.866852
109,37.239580,127.094128,1749.377651,1865.419774,40,69975.106034
...,...,...,...,...,...,...
501,37.062698,127.047770,1895.194907,1509.302856,32,60646.237018
112,37.101601,126.920339,1575.314791,1345.840700,40,63012.591622
680,37.289144,126.736108,1874.936004,1516.976770,32,59997.952120
476,37.019957,127.140616,1856.520904,1225.581824,24,44556.501698
